In [78]:
import numpy as np
import pandas as pd

# 0. 아래 명령어로 판다스 버전 정보 출력하기

In [79]:
# 0번 문제

print(pd.__version__)

2.2.3


# 1. 기상청 (https://data.kma.go.kr) 에서 다음 정보들을 다운받아 지정된 파일로 저장한다.(총 8개 파일)

# 2. Pandas를 이용해 8개의 데이터 파일을 읽어 데이터프레임을 만든다. 데이터프레임 이름은 확장자명(.csv)를 제외한 파일명과 동일하게 한다. 단, 다운받은 파일을 다른 프로그램으로 수정하지 않고 그대로 사용해서 데이터프레임을 만들어야 함.


In [80]:
# 2번 문제

import pandas as pd

def load_csv(filename, encoding='euc-kr', skiprows=7):
    df = pd.read_csv(filename, encoding=encoding, skiprows=skiprows)
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.strip()
    return df
"""
지정된 csv 파일을 읽어와서 데이터프레임으로 변환
euc-kr 인코딩으로 한글 파일을 처리
헤더 위의 설명줄 skiprow 제거
문자열 컬럼의 공백 및 탭 문자 제거
"""
BS_RAIN = load_csv('BS_RAIN.csv')
BS_TEMP = load_csv('BS_TEMP.csv')
DG_RAIN = load_csv('DG_RAIN.csv')
DG_TEMP = load_csv('DG_TEMP.csv')
GJ_RAIN = load_csv('GJ_RAIN.csv')
GJ_TEMP = load_csv('GJ_TEMP.csv')
SO_RAIN = load_csv('SO_RAIN.csv')
SO_TEMP = load_csv('SO_TEMP.csv')
# 앞선 가공한 데이터 불러옴


In [81]:
# 2번 문제
SO_TEMP.head(3)

,날짜,지점,평균기온(℃),최저기온(℃),최고기온(℃)
0,1990-01-01,108,-1.7,-4.0,1.8
1,1990-01-02,108,-3.5,-5.7,-0.9
2,1990-01-03,108,-5.0,-7.2,-1.7


In [82]:
# 2번 문제
DG_TEMP.head(3)

,날짜,지점,평균기온(℃),최저기온(℃),최고기온(℃)
0,1990-01-01,143,0.1,-3.8,5.1
1,1990-01-02,143,0.7,-1.8,5.3
2,1990-01-03,143,-1.6,-3.9,2.8


# 3. 기온 정보를 가지고 있는 4개 데이터프레임의 컬럼 인덱스를 다음과 같은 형식으로 변경한다.

In [83]:
# 3번 문제

# SO_TEMP (서울)
SO_TEMP.columns = ['날짜', '서울지점', '서울평균', '서울최저', '서울최고']

# DG_TEMP (대구)
DG_TEMP.columns = ['날짜', '대구지점', '대구평균', '대구최저', '대구최고']

# GJ_TEMP (광주)
GJ_TEMP.columns = ['날짜', '광주지점', '광주평균', '광주최저', '광주최고']

# BS_TEMP (부산)
BS_TEMP.columns = ['날짜', '부산지점', '부산평균', '부산최저', '부산최고']

In [84]:
# 3번 문제
SO_TEMP.head(1)

,날짜,서울지점,서울평균,서울최저,서울최고
0,1990-01-01,108,-1.7,-4.0,1.8


In [85]:
# 3번 문제
DG_TEMP.head(1)

,날짜,대구지점,대구평균,대구최저,대구최고
0,1990-01-01,143,0.1,-3.8,5.1


# 4. 4개 도시의 기온 정보 데이터 프레임을 모아서 ALL_TEMP 데이터프레임을 만든다.

In [86]:
# 4번 문제
# '날짜' 컬럼 기준으로 4개 도시 데이터프레임 병합
ALL_TEMP = pd.concat(
    [SO_TEMP.set_index('날짜'),
     DG_TEMP.set_index('날짜'),
     GJ_TEMP.set_index('날짜'),
     BS_TEMP.set_index('날짜')],
    axis=1
)

In [87]:
# 4번 문제
ALL_TEMP.head(2)

,서울지점,서울평균,서울최저,서울최고,대구지점,대구평균,대구최저,대구최고,광주지점,광주평균,광주최저,광주최고,부산지점,부산평균,부산최저,부산최고
날짜,,,,,,,,,,,,,,,,
1990-01-01,108,-1.7,-4.0,1.8,143,0.1,-3.8,5.1,156,1.5,-3.2,7.7,159,3.8,-1.0,11.3
1990-01-02,108,-3.5,-5.7,-0.9,143,0.7,-1.8,5.3,156,0.3,-0.7,4.7,159,2.1,-0.3,6.0


# 5. 4개 도시(서울,대구,광주,부산)의 각 년도별 최고 기온, 최저 기온을 구해 ANU_TEMP 데이터프레임을 만든다

In [88]:
# 5번 문제

# 날짜를 datetime 형식으로 변환
ALL_TEMP.index = pd.to_datetime(ALL_TEMP.index)

# 연도 컬럼 추가
ALL_TEMP['연도'] = ALL_TEMP.index.year

# 새로운 빈 데이터프레임 생성
ANU_TEMP = pd.DataFrame()

# 연도별로 반복하면서 최고/최저 계산
for year in sorted(ALL_TEMP['연도'].unique()):
    # unique  : [1990, 1990, 1990, 1991, 1991, 1992, 1992, 1992] -> array([1990, 1991, 1992])
    # 해당 연도 데이터만 선택
    data = ALL_TEMP[ALL_TEMP['연도'] == year]

    # 각 도시별 최고기온, 최저기온 계산
    seoul_h = data['서울최고'].max()
    daegu_h = data['대구최고'].max()
    gwangju_h = data['광주최고'].max()
    busan_h = data['부산최고'].max()

    seoul_l = data['서울최저'].min()
    daegu_l = data['대구최저'].min()
    gwangju_l = data['광주최저'].min()
    busan_l = data['부산최저'].min()

    # 한 행(row)으로 묶어서 새로운 DF에 추가
    row = pd.DataFrame({
        '서울최고': [seoul_h],
        '대구최고': [daegu_h],
        '광주최고': [gwangju_h],
        '부산최고': [busan_h],
        '서울최저': [seoul_l],
        '대구최저': [daegu_l],
        '광주최저': [gwangju_l],
        '부산최저': [busan_l]
    }, index=[year])

    # 기존 ANU_TEMP 이어서 붙임
    ANU_TEMP = pd.concat([ANU_TEMP, row])

In [89]:
# 5번 문제
ANU_TEMP.head(3)

,서울최고,대구최고,광주최고,부산최고,서울최저,대구최저,광주최저,부산최저
1990,35.5,38.5,36.0,34.3,-17.1,-11.8,-12.5,-10.4
1991,34.7,35.0,35.8,33.3,-15.0,-12.4,-11.7,-10.8
1992,32.8,37.8,33.8,32.6,-9.7,-8.2,-6.2,-4.5


# 6. 각 년도별로 최저 기온이 가장 높은 도시를 구한다.

In [109]:
# 6번 문제

# 4개 도시의 최고기온 열만 선택
high_temp_cols = ['서울최고', '대구최고', '광주최고', '부산최고']

# 각 연도별로 최고기온이 가장 높은 도시 구하기
result = ANU_TEMP[high_temp_cols].idxmax(axis=1)

result.index.name = '날짜'
result

날짜
1990    대구최고
1991    광주최고
1992    대구최고
1993    대구최고
1994    대구최고
1995    대구최고
1996    대구최고
1997    대구최고
1998    대구최고
1999    대구최고
2000    서울최고
2001    대구최고
2002    대구최고
2003    대구최고
2004    대구최고
2005    대구최고
2006    대구최고
2007    대구최고
2008    대구최고
2009    대구최고
2010    대구최고
2011    대구최고
2012    광주최고
2013    대구최고
2014    대구최고
2015    대구최고
2016    대구최고
2017    대구최고
2018    서울최고
2019    대구최고
2020    대구최고
2021    대구최고
2022    대구최고
2023    대구최고
dtype: object

# 7. 각 도시별로 최고 기온이 가장 높았던 해의 횟수를 구하고, 내림차순으로 정렬해서 보여준다.

In [113]:
# 7번 문제
# 각 연도별 최고기온이 가장 높은 도시 찾기
max_ = ANU_TEMP[['서울최고', '대구최고', '광주최고', '부산최고']].idxmax(axis=1)
count = max_.value_counts() # 도시별 횟수 카운트
count = count.sort_values(ascending=False) # 내림차순으로 정렬
count

대구최고    30
광주최고     2
서울최고     2
Name: count, dtype: int64

# 8. 각 년도별로 최저 기온이 가장 낮은 도시를 구한다.

In [117]:
# 8번 문제

# 4개 도시의 최저기온 열만 선택
low_temp_cols = ['서울최저', '대구최저', '광주최저', '부산최저']

# 각 연도별로 최저기온이 가장 높은 도시 구하기
result = ANU_TEMP[low_temp_cols].idxmin(axis=1)

result.index.name = '날짜'
print(result)

날짜
1990    서울최저
1991    서울최저
1992    서울최저
1993    서울최저
1994    서울최저
1995    서울최저
1996    서울최저
1997    서울최저
1998    서울최저
1999    서울최저
2000    서울최저
2001    서울최저
2002    서울최저
2003    서울최저
2004    서울최저
2005    서울최저
2006    서울최저
2007    서울최저
2008    서울최저
2009    서울최저
2010    서울최저
2011    서울최저
2012    서울최저
2013    서울최저
2014    서울최저
2015    서울최저
2016    서울최저
2017    서울최저
2018    서울최저
2019    서울최저
2020    서울최저
2021    서울최저
2022    서울최저
2023    서울최저
dtype: object


# 9. 각 도시별로 최저 기온이 가장 낮았던 해의 횟수를 구하고, 내림차순으로 정렬해서 보여준다.

In [134]:
# 9번 문제
# 각 연도별 최저기온이 가장 낮은 도시 찾기
min_ = ANU_TEMP[['서울최저', '대구최저', '광주최저', '부산최저']].idxmin(axis=1)
count = min_.value_counts() # 도시별 횟수 카운트
count = count.sort_values(ascending=False) # 내림차순으로 정렬
count

서울최저    34
Name: count, dtype: int64

# 10. 각 도시별 7,8,9월의 평균 기온(여름평균)을 구하고, 또한 12,1,2월(겨울평균)의 평균 기온을 구해 그 차이(기온차)를 계산하고 오름차순으로 정렬해서 보여준다.

In [135]:
# 10번 문제

ALL_TEMP.index = pd.to_datetime(ALL_TEMP.index)
ALL_TEMP['월'] = ALL_TEMP.index.month
avg_cols = ['서울평균', '대구평균', '광주평균', '부산평균'] 
summer = ALL_TEMP[ALL_TEMP['월'].isin([7, 8, 9])][avg_cols].mean() # 여름 평균 계산
winter = ALL_TEMP[ALL_TEMP['월'].isin([12, 1, 2])][avg_cols].mean() # 겨울 평균 계산
diff = summer - winter # 계절 차이 계산
# 수집한 결과를 하나의 데이터프레임으로 생성
result = pd.DataFrame({
    '여름평균': summer,
    '겨울평균': winter,
    '기온차': diff
})
result = result.sort_values(by='기온차', ascending=True) # 오름차순으로 정렬
result

,여름평균,겨울평균,기온차
부산평균,24.515281,4.893709,19.621572
광주평균,24.943670,2.414374,22.529296
대구평균,25.014711,2.378422,22.636288
서울평균,24.475607,-0.396773,24.872381


# 11. 4개 도시의 강수량 데이터프레임을 모아 ALL_RAIN 데이터프레임을 만든다. 단, 컬럼의인덱스는 다음과 같게 한다. (서울강수량, 대구강수량, 광주강수량, 부산강수량)

In [136]:
# 11번 문제

# 서울
seoul_rain = SO_RAIN.set_index('날짜').iloc[:, -1].rename('서울강수량')
# .iloc[:,-1] 마지막 열 = 실제 강수량만 파싱

# 대구
daegu_rain = DG_RAIN.set_index('날짜').iloc[:, -1].rename('대구강수량')
# 광주
gwangju_rain = GJ_RAIN.set_index('날짜').iloc[:, -1].rename('광주강수량')
# 부산
busan_rain = BS_RAIN.set_index('날짜').iloc[:, -1].rename('부산강수량')

# 네 개를 날짜 기준으로 가로로 합치기
ALL_RAIN = pd.concat(
    [seoul_rain, daegu_rain, gwangju_rain, busan_rain],
    axis=1
)

ALL_RAIN.head(3)

,서울강수량,대구강수량,광주강수량,부산강수량
날짜,,,,
1990-01-01,0.0,0.0,0.4,0.0
1990-01-02,0.1,NaN,0.4,NaN
1990-01-03,NaN,NaN,0.5,NaN


# 12. 각 도시의 각 년도별로 강수량 총합을 구해 ANU_RAIN 데이터프레임을 만든다.

In [137]:
# 12번 문제

# 날짜 인덱스를 datetime으로 변환
ALL_RAIN.index = pd.to_datetime(ALL_RAIN.index)

# 연도 컬럼 추가
ALL_RAIN['연도'] = ALL_RAIN.index.year

# 연도별 강수량 합계 계산
ANU_RAIN = ALL_RAIN.groupby('연도')[['서울강수량', '대구강수량', '광주강수량', '부산강수량']].sum()

ANU_RAIN

,서울강수량,대구강수량,광주강수량,부산강수량
연도,,,,
1990,2355.5,1239.7,1484.2,1270.9
1991,1158.2,1296.2,1480.5,2167.1
1992,1454.9,811.7,1098.1,1308.9
1993,1292.7,1181.7,1372.1,1615.3
1994,1055.8,567.5,776.4,959.9
1995,1598.6,728.8,764.4,1005.7
1996,1256.6,888.5,1268.8,1289.6
1997,1210.2,1122.2,1479.2,1598.1
1998,2349.1,1381.8,1943.0,2028.8


# 13. 각 년도별로 최고 강수량을 기록한 도시를 구한다

In [143]:
# 13번 문제

# 각 연도별로 가장 많은 강수량을 기록한 도시 찾기
result = ANU_RAIN[['서울강수량', '대구강수량', '광주강수량', '부산강수량']].idxmax(axis=1)

# 결과 출력
result

날짜
1990    서울강수량
1991    부산강수량
1992    서울강수량
1993    부산강수량
1994    서울강수량
1995    서울강수량
1996    부산강수량
1997    부산강수량
1998    서울강수량
1999    부산강수량
2000    광주강수량
2001    서울강수량
2002    부산강수량
2003    부산강수량
2004    광주강수량
2005    부산강수량
2006    서울강수량
2007    광주강수량
2008    서울강수량
2009    부산강수량
2010    서울강수량
2011    서울강수량
2012    부산강수량
2013    서울강수량
2014    부산강수량
2015    부산강수량
2016    부산강수량
2017    서울강수량
2018    부산강수량
2019    부산강수량
2020    부산강수량
2021    부산강수량
2022    서울강수량
2023    부산강수량
dtype: object

# 14. 각 도시별로 강수량 총합이 가장 높았던 해의 횟수를 구하고, 내림차순으로 정렬해서 보여준다.

In [142]:
# 14번 문제
# 13번 result를 가져와 횟수 계산
rain_cnt = result.value_counts()
rain_cnt = rain_cnt.sort_values(ascending=False)
rain_cnt

도시   
부산강수량    18
서울강수량    13
광주강수량     3
Name: count, dtype: int64

# 15. 각 년도별로 최저 강수량을 기록한 도시를 구한다.

In [167]:
# 15번 문제
# 13번 문제와 동일
min_rain = ANU_RAIN[['서울강수량', '대구강수량', '광주강수량', '부산강수량']].idxmin(axis=1)
min_rain


날짜
1990    대구강수량
1991    서울강수량
1992    대구강수량
1993    대구강수량
1994    대구강수량
1995    대구강수량
1996    대구강수량
1997    대구강수량
1998    대구강수량
1999    대구강수량
2000    대구강수량
2001    대구강수량
2002    대구강수량
2003    대구강수량
2004    대구강수량
2005    대구강수량
2006    대구강수량
2007    대구강수량
2008    대구강수량
2009    대구강수량
2010    대구강수량
2011    광주강수량
2012    대구강수량
2013    대구강수량
2014    서울강수량
2015    서울강수량
2016    서울강수량
2017    대구강수량
2018    서울강수량
2019    서울강수량
2020    대구강수량
2021    대구강수량
2022    대구강수량
2023    대구강수량
dtype: object

# 16. 각 도시별로 강수량 총합이 가장 낮았던 해의 회수를 구하고, 내림차순으로 정렬해서 보여준다.

In [151]:
# 16번 문제
# 14번 문제와 동일
min_cnt = min_rain.value_counts()
min_cnt = min_cnt.sort_values(ascending=False)
min_cnt


대구강수량    27
서울강수량     6
광주강수량     1
Name: count, dtype: int64

# 17. 각 도시별로 강수량이 가장 높았던 해와 낮았던 해를 구한다.

In [159]:
# 17번 문제 
# 각 도시의 최대 강수량 탐색
max_year = ANU_RAIN[['서울강수량', '대구강수량', '광주강수량', '부산강수량']].idxmax()
# 각 도시 최소 강수량 탐색
min_year = ANU_RAIN[['서울강수량', '대구강수량', '광주강수량', '부산강수량']].idxmin()

print(max_year)
print(min_year)

서울강수량    1990
대구강수량    2003
광주강수량    2023
부산강수량    1999
dtype: int32
서울강수량    2015
대구강수량    1994
광주강수량    1995
부산강수량    1994
dtype: int32


# 18. 각 도시의 기온과 강수량 정보를 모두 통합해 ALL_WEATHER 데이터프레임을 만드시오.

In [162]:
# 18번 문제 
# 앞선 ALL_TEMP, ALL_RAIN 이어 붙이기
ALL_WEATHER = pd.concat([ALL_TEMP, ALL_RAIN], axis=1)
ALL_WEATHER.head()

,서울지점,서울평균,서울최저,서울최고,대구지점,대구평균,대구최저,대구최고,광주지점,광주평균,...,부산평균,부산최저,부산최고,연도,월,서울강수량,대구강수량,광주강수량,부산강수량,연도
날짜,,,,,,,,,,,,,,,,,,,,,
1990-01-01,108,-1.7,-4.0,1.8,143,0.1,-3.8,5.1,156,1.5,...,3.8,-1.0,11.3,1990,1,0.0,0.0,0.4,0.0,1990
1990-01-02,108,-3.5,-5.7,-0.9,143,0.7,-1.8,5.3,156,0.3,...,2.1,-0.3,6.0,1990,1,0.1,NaN,0.4,NaN,1990
1990-01-03,108,-5.0,-7.2,-1.7,143,-1.6,-3.9,2.8,156,-0.6,...,-0.2,-3.5,4.4,1990,1,NaN,NaN,0.5,NaN,1990
1990-01-04,108,-3.3,-8.9,2.1,143,-0.8,-4.6,5.3,156,-0.3,...,0.9,-3.4,6.2,1990,1,NaN,NaN,NaN,NaN,1990
1990-01-05,108,0.7,-2.4,2.1,143,3.0,-2.7,8.1,156,3.9,...,4.4,0.0,10.1,1990,1,3.2,0.4,1.2,NaN,1990


# 19. 각 도시별로 강수량이 가장 높은 날 100개씩 구하시오.

In [166]:
# 19번 문제
# 서울
seoul_rain = ALL_RAIN[['서울강수량']]
seoul_rain_sorted = seoul_rain.sort_values(by='서울강수량', ascending=False) # 내림차순 정렬
seoul_top100 = seoul_rain_sorted.head(100) # 상위 100
seoul_temp = ALL_TEMP.loc[seoul_top100.index, ['서울최저', '서울최고']]
seoul_result = pd.concat([seoul_top100, seoul_temp], axis=1) # 기온 + 강수량
print(seoul_result)

# 대구
daegu_rain = ALL_RAIN[['대구강수량']]
daegu_rain_sorted = daegu_rain.sort_values(by='대구강수량', ascending=False)
daegu_top100 = daegu_rain_sorted.head(100)
daegu_temp = ALL_TEMP.loc[daegu_top100.index, ['대구최저', '대구최고']]
daegu_result = pd.concat([daegu_top100, daegu_temp], axis=1)
print(daegu_result)

# 광주
gwangju_rain = ALL_RAIN[['광주강수량']]
gwangju_rain_sorted = gwangju_rain.sort_values(by='광주강수량', ascending=False)
gwangju_top100 = gwangju_rain_sorted.head(100)
gwangju_temp = ALL_TEMP.loc[gwangju_top100.index, ['광주최저', '광주최고']]
gwangju_result = pd.concat([gwangju_top100, gwangju_temp], axis=1)
print(gwangju_result)

# 부산
busan_rain = ALL_RAIN[['부산강수량']]
busan_rain_sorted = busan_rain.sort_values(by='부산강수량', ascending=False)
busan_top100 = busan_rain_sorted.head(100)
busan_temp = ALL_TEMP.loc[busan_top100.index, ['부산최저', '부산최고']]
busan_result = pd.concat([busan_top100, busan_temp], axis=1)
print(busan_result)

            서울강수량  서울최저  서울최고
날짜                           
1998-08-08  332.8  23.6  25.5
2011-07-27  301.5  22.3  26.2
2001-07-15  273.4  21.3  27.3
1999-08-02  261.6  19.6  23.4
2010-09-21  259.5  17.9  24.9
...           ...   ...   ...
1998-06-25   85.7  20.8  24.2
2012-09-17   85.5  16.0  18.4
2001-08-13   84.5  19.4  23.6
2013-07-22   84.5  24.1  29.4
1993-07-11   84.2  20.7  25.0

[100 rows x 3 columns]
            대구강수량  대구최저  대구최고
날짜                           
1998-09-30  225.8  18.6  20.2
2003-09-12  187.0  20.8  25.7
2020-08-08  168.2  22.9  25.0
2011-07-09  168.0  21.8  27.6
2004-08-18  161.0  21.6  26.6
...           ...   ...   ...
2004-08-23   60.0  18.2  23.8
2019-06-26   60.0  18.3  22.3
1999-07-02   59.7  18.8  20.9
2001-06-25   59.2  21.1  30.0
2002-07-19   59.0  20.7  26.6

[100 rows x 3 columns]
            광주강수량  광주최저  광주최고
날짜                           
2004-08-18  322.5  20.4  27.1
2020-08-07  259.5  23.2  26.0
2020-08-08  255.5  23.0  27.1
1997-08-04  216.5  21.

# 20. 각 도시별 최고 강수량 100일의 최고 기온과 최저 기온의 평균을 구하시오

In [173]:
# 20번 문제

data = {
    '서울최고': seoul_result['서울최고'].mean(), # 평균값 계산
    '서울최저': seoul_result['서울최저'].mean(),
    '대구최고': daegu_result['대구최고'].mean(),
    '대구최저': daegu_result['대구최저'].mean(),
    '광주최고': gwangju_result['광주최고'].mean(),
    '광주최저': gwangju_result['광주최저'].mean(),
    '부산최고': busan_result['부산최고'].mean(),
    '부산최저': busan_result['부산최저'].mean()
}

series = pd.Series(data) # 데이터를 가지고 시리즈 생성
series

서울최고    24.834000
서울최저    20.545455
대구최고    25.149000
대구최저    20.144000
광주최고    26.109000
광주최저    21.263000
부산최고    23.504000
부산최저    19.748000
dtype: float64